### Лабораторная работа №4
####  Исследование абсолютно-оптимальных рекуррентных алгоритмов

Для оценки эффективности абсолютно-оптимальных алгоритмов рассмотрим задачу идентификации параметров линейного регрессионного объекта вида:
\begin{equation}y(i)=с_{0}+с_{1}u_{1}(i)+...+с_{4}u_{4}(i)+\eta(i)\end{equation}

Пусть шум измерений $\eta(i)$ имеет распределение Коши:
\begin{equation*}
f(\eta)=\frac{1}{\pi s}\frac{1}{(1+(\eta/s)^2)}
\end{equation*}

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

In [ ]:
# параметры объекта - Вариант 3
c_params = [1.5, 2.5, -3.5, 4.5, 5.0]

In [ ]:
N = 200
s = 0.1
noise = st.cauchy.rvs(loc=0, scale=s, size=N)

In [ ]:
avg_u1, var_u1 = 0, 10
avg_u2, var_u2 = 1, 50
avg_u3, var_u3 = 0, 1
avg_u4, var_u4 = 5, 50

In [ ]:
# моделирование входов объекта
np.random.seed(42)
u1 = np.random.normal(avg_u1, var_u1, size=N)
u2 = np.random.normal(avg_u2, var_u2, size=N)
u3 = np.random.normal(avg_u3, var_u3, size=N)
u4 = np.random.normal(avg_u4, var_u4, size=N)
u_input = [u1, u2, u3, u4]

In [ ]:
# моделирование выхода объекта
y = c_params[0] + sum([c*u for c, u in zip(c_params[1::], u_input)]) + noise

In [ ]:
plt.plot(y)
plt.show()

#### Формирование алгоритма идентификации

В зависимости от того, насколько хорошо изучен «объект» идентификации, могут возникнуть следующие ситуации:

1. Объект изучен хорошо, правильно определена плотность распределения шума, т.е. принятая и реальная плотности распределения совпадают и на основе принятой функции распределения формируется оптимальная функция потерь.


2. Объект изучен плохо. При этом, как правило, считают, что плотность распределения шума соответствует нормальному закону распределения, хотя на самом деле(в данном случаем, см. описание объекта выше) шум имеет распределение Коши. Таким образом, опираясь на ложную гипотезу о нормальном распределении шума, в качестве функции потерь выбирается квадратичная функция.

Предлагается реализовать абсолютно-оптимальный рекуррентный алгоритм оценивания параметров объекта для каждой из ситуаций. Для   оценки   эффективности использования абсолютно-оптимальных  рекуррентных  алгоритмов  проводилось сравнение сглаженной ошибки оценки параметров объекта,определенных когда объект изучили хорошо и когда плохо, при  различных  значениях  параметра  распределения Коши — s.

#### В первом случае рекуррентный алгоритм принимает вид:
\begin{equation}\hat{\vec{c}}(i)=\hat{\vec{c}}(i-1)+Г(i)\frac{2(y(i)-\hat{c}_{0}(i-1)u_{0}(i)-...-\hat{c}_{4}(i-1)u_{4}(i))}{s^2+(y(i)-\hat{c}_{0}(i-1)u_{0}(i)-...-\hat{c}_{4}(i-1)u_{4}(i))^2}\vec{z}(i) \\
Г(i)=Г(i-1)-\frac{Г(i-1)\vec{z}(i)\vec{z}(i)^TГ(i-1)}{2s^2+\vec{z}^T(i)Г(i-1)\vec{z}(i)} \\
Г(0)=\lambda I, \lambda=0.1\\
\hat{\vec{c}}(0)=\vec{c}_{0}\end{equation}

In [ ]:
c_result_1 = [[c_params]]
c_result_2 = [[c_params]]
lambda_ = 0.1
u_transposed = [*zip(*u_input)]

In [ ]:
G = lambda_ * np.eye(5)
for i in range(N):
    z = np.asmatrix(([1] + [input[i] for input in u_input])).T

    G_tmp = (G * z * z.T * G)/(2 * s**2 + z.T * G * z)
    G = G - G_tmp

    c_tmp = y[i] - sum([-1*c*u for c, u in zip(c_params[1::], u_transposed[i])])
    c = c_result_1[i] + 2 * G * z * c_tmp/(s**2 + c_tmp**2)

    c_result_1.append(c)
print(c_result_1[N][-1])

#### Во втором случае рекуррентный алгоритм принимает вид:
\begin{equation}\hat{\vec{c}}(i)=\hat{\vec{c}}(i-1)+Г(i)\vec{z}(i)(y(i)-\hat{c}_{0}(i-1)u_{0}(i)-...-\hat{c}_{4}(i-1)u_{4}(i)) \\
Г(i)=Г(i-1)-\frac{Г(i-1)\vec{z}(i)\vec{z}(i)^TГ(i-1)}{1+\vec{z}^T(i)Г(i-1)\vec{z}(i)} \\
Г(0)=\lambda I , \lambda=0.1    \\
\hat{\vec{c}}(0)=\vec{c}_{0}\end{equation}

In [ ]:
G = lambda_ * np.eye(5)
for i in range(N):
    z = np.asmatrix(([1] + [input[i] for input in u_input])).T

    G_tmp = (G * z * z.T * G)/(2 * s**2 + z.T * G * z)
    G = G - G_tmp

    c_tmp = y[i] - sum([-1*c*u for c, u in zip(c_params[1::], u_transposed[i])])
    c = c_result_2[i] + G * z * c_tmp

    c_result_2.append(c)
print(c_result_2[N][-1])

Сглаженная ошибка оценки вычисляется по формуле:
\begin{equation*}
err_{сгл}=\sqrt{\frac{\sum_{j=1}^{10}\sum_{k=0}^4(\hat{c}_{k}(i-j)-c_{k})^2}{10}} ; i=10,11,12...
\end{equation*}

In [ ]:
root_mse_1, root_mse_2 = [], []
for i in range(10,N):
    root1, root2 = 0, 0
    for j in range(1,11):
        for k in range(5):
            root1 += (np.squeeze(np.asarray(c_result_1[i - j][-1]))[k] - c_params[k])**2
            root2 += (np.squeeze(np.asarray(c_result_2[i - j][-1]))[k] - c_params[k])**2
    root_mse_1.append((root1/10)**0.5)
    root_mse_2.append((root2/10)**0.5)
plt.plot(root_mse_2, label='Объект изучен плохо')
plt.plot(root_mse_1, label='Объект изучен хорошо')
plt.legend()